# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import holoviews as hv
# import hvplot.pandas
import pandas as pd
import requests

import warnings
warnings.filterwarnings("ignore")
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,aasiaat,68.7098,-52.8699,2.89,92,100,0.59,GL,1689531991
1,1,wuchang,44.9143,127.1500,17.61,99,100,3.35,CN,1689531992
2,2,ust-nera,64.5667,143.2000,9.24,85,44,3.59,RU,1689531992
3,3,edinburgh of the seven seas,-37.0676,-12.3116,13.13,78,17,13.00,SH,1689531992
4,4,wad medani,14.4012,33.5199,33.44,37,23,4.93,SD,1689531993


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_cities = city_data_df.hvplot.points("Lat", "Lng", geo=True, size="Humidity")

# Display the map
map_plot_cities

AttributeError: 'DataFrame' object has no attribute 'hvplot'

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0),:]
city_data_df
# Drop any rows with null values
city_data_df.dropna(how="any")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
72,72,swakopmund,-22.6833,14.5333,22.47,75,0,1.34,NaN,1689532024
106,106,afaahiti,-17.7500,-149.2833,23.93,64,0,2.27,PF,1689532041
210,210,kastsyukowka,52.5387,30.9173,23.00,60,0,2.00,BY,1689532097
341,341,laojunmiao,39.8333,97.7333,23.78,23,0,3.53,CN,1689532164
365,365,saint-francois,46.4154,3.9054,23.15,49,0,3.09,FR,1689532176


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]
hotel_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
72,swakopmund,NaN,-22.6833,14.5333,75,
106,afaahiti,PF,-17.7500,-149.2833,64,
210,kastsyukowka,BY,52.5387,30.9173,60,
341,laojunmiao,CN,39.8333,97.7333,23,
365,saint-francois,FR,46.4154,3.9054,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
swakopmund - nearest hotel: Amanpuri Travellers Lodge
afaahiti - nearest hotel: Omati Lodge
kastsyukowka - nearest hotel: "экспрэс"
laojunmiao - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
pacific grove - nearest hotel: Pacific Grove Inn
walvis bay - nearest hotel: Atlantic
shevchenko - nearest hotel: Premier Hotel Abri
bauru - nearest hotel: Biazi Plaza Hotel
vylkove - nearest hotel: Готель Венеція


,City,Country,Lat,Lng,Humidity,Hotel Name
72,swakopmund,NaN,-22.6833,14.5333,75,Amanpuri Travellers Lodge
106,afaahiti,PF,-17.7500,-149.2833,64,Omati Lodge
210,kastsyukowka,BY,52.5387,30.9173,60,"""экспрэс"""
341,laojunmiao,CN,39.8333,97.7333,23,No hotel found
365,saint-francois,FR,46.4154,3.9054,49,Chez Lily
373,pacific grove,US,36.6177,-121.9166,79,Pacific Grove Inn
497,walvis bay,NaN,-22.9575,14.5053,35,Atlantic
533,shevchenko,UA,48.5000,35.0000,47,Premier Hotel Abri
558,bauru,BR,-22.3147,-49.0606,38,Biazi Plaza Hotel
565,vylkove,UA,45.4021,29.5860,64,Готель Венеція


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE